# Unwrap interferograms with snaphu using scaled speckle tracking offsets for coarse phase estimate

In [1]:
import xarray as xr
import rasterio as rio
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal, gdal_array
from skimage import data, filters
from scipy.interpolate import interpn
import os
from glob import glob
from datetime import datetime

In [2]:
def prep_inputs(igram_dir, speckle_dir, tbaseline):
    '''
    - open interferogram, coherence, and speckle tracking product
    - resample and pad speckle tracking offsets to match interferogram
    - filter speckle tracking product, convert to phase, and scale to temporal baseline
    
    '''
    veloc_fn = f'{speckle_dir}/velocityRg.h5'
    igram_fn = f'{igram_dir}/filt_fine.int'
    cor_fn = f'{igram_dir}/filt_fine.cor'

    igram_ds = gdal.Open(igram_fn, gdal.GA_ReadOnly)
    igram = igram_ds.GetRasterBand(1).ReadAsArray()
    cor_ds = gdal.Open(cor_fn, gdal.GA_ReadOnly)
    cor = cor_ds.GetRasterBand(1).ReadAsArray()

    ds = xr.open_dataset(veloc_fn)

    non_zero_rows = np.any(igram != 0, axis=1)
    non_zero_cols = np.any(igram != 0, axis=0)
    igram_crop = igram[non_zero_rows][:, non_zero_cols]
    cor_crop = cor[non_zero_rows][:, non_zero_cols]

    veloc = ds.velocity.values
    non_zero_rows = np.any(veloc != 0, axis=1)
    non_zero_cols = np.any(veloc != 0, axis=0)

    veloc_crop = veloc[non_zero_rows][:, non_zero_cols]

    # Find rows and columns with all zeros and remove them in each direction
    row_sum = np.sum(igram, axis=1)
    col_sum = np.sum(igram, axis=0)
    top_padding = 0
    bottom_padding = 0
    left_padding = 0
    right_padding = 0
    
    # Determine top padding
    while row_sum[top_padding] == 0:
        top_padding += 1
    
    # Determine bottom padding
    while row_sum[-1 - bottom_padding] == 0:
        bottom_padding += 1
    
    # Determine left padding
    while col_sum[left_padding] == 0:
        left_padding += 1
    
    # Determine right padding
    while col_sum[-1 - right_padding] == 0:
        right_padding += 1

    # interpolate to match interferogram dimensions
    x_coords = np.linspace(0, veloc_crop.shape[1]-1, veloc_crop.shape[1])
    y_coords = np.linspace(0, veloc_crop.shape[0]-1, veloc_crop.shape[0])
    
    # Create a mesh grid for the interferogram dimensions
    x_coords_new, y_coords_new = np.meshgrid(
        np.linspace(x_coords.min(), x_coords.max(), igram_crop.shape[1]),
        np.linspace(y_coords.min(), y_coords.max(), igram_crop.shape[0])
    )
    
    # Perform bilinear interpolation using scipy.interpolate.interpn
    veloc = interpn((y_coords, x_coords), veloc_crop, (y_coords_new, x_coords_new), method="linear")

    # filter to remove some noise
    veloc_filtered = filters.butterworth(veloc,
                                         cutoff_frequency_ratio=0.35, #between 0 and 0.5
                                         order=2,
                                         high_pass=False)
    
    # convert to phase for simulated 12 day period
    veloc_unwrapped = (veloc_filtered-0.6)*(tbaseline/365)*(12.5663706/0.05546576) # 0.6 to set ref point
    # pad to original interferogram dimensions
    veloc_unwrapped_full = np.pad(veloc_unwrapped, ((top_padding, bottom_padding), (left_padding, right_padding)), mode='constant')

    # crop to aoi
    veloc_unwrapped_aoi = veloc_unwrapped_full[6400:7400, 2400:3400]
    igram_aoi = igram[6400:7400, 2400:3400]
    cor_aoi = cor[6400:7400, 2400:3400]

    # save cropped speckle tracking displacement
    veloc_unwrapped_aoi = np.nan_to_num(veloc_unwrapped_aoi, nan=0)
    veloc_unwrapped_aoi = veloc_unwrapped_aoi.astype(np.float32)
    output_file = f'{igram_dir}/veloc_unwrapped_aoi' 
    driver_format = "ISCE"    # Specify the GDAL format for the output (GeoTIFF in this example)
    rows, cols = veloc_unwrapped_aoi.shape    # Get the number of rows and columns from the NumPy array
    data_type = gdal_array.NumericTypeCodeToGDALTypeCode(veloc_unwrapped_aoi.dtype)  # Convert NumPy data type to GDAL data type
    driver = gdal.GetDriverByName(driver_format)
    output_ds = driver.Create(output_file, cols, rows, 1, data_type)
    band = output_ds.GetRasterBand(1)  
    band.WriteArray(veloc_unwrapped_aoi)
    output_ds = None

    # save cropped interferogram
    igram_aoi = np.nan_to_num(igram_aoi, nan=0)
    igram_aoi = igram_aoi.astype(np.complex64)
    output_file = f'{igram_dir}/filt_fine_aoi.int' 
    driver_format = "ISCE"    # Specify the GDAL format for the output (GeoTIFF in this example)
    rows, cols = igram_aoi.shape    # Get the number of rows and columns from the NumPy array
    data_type = gdal_array.NumericTypeCodeToGDALTypeCode(igram_aoi.dtype)  # Convert NumPy data type to GDAL data type
    driver = gdal.GetDriverByName(driver_format)
    output_ds = driver.Create(output_file, cols, rows, 1, data_type)
    band = output_ds.GetRasterBand(1)  
    band.WriteArray(igram_aoi)
    output_ds = None

    # save cropped coherence
    cor_aoi = np.nan_to_num(cor_aoi, nan=0)
    cor_aoi = cor_aoi.astype(np.float32)
    output_file = f'{igram_dir}/filt_fine_aoi.cor' 
    driver_format = "ISCE"    # Specify the GDAL format for the output (GeoTIFF in this example)
    rows, cols = cor_aoi.shape    # Get the number of rows and columns from the NumPy array
    data_type = gdal_array.NumericTypeCodeToGDALTypeCode(cor_aoi.dtype)  # Convert NumPy data type to GDAL data type
    driver = gdal.GetDriverByName(driver_format)
    output_ds = driver.Create(output_file, cols, rows, 1, data_type)
    band = output_ds.GetRasterBand(1) 
    band.WriteArray(cor_aoi)
    output_ds = None

    # # save coherence mask 
    # byte_mask = (np.abs(veloc_unwrapped_aoi)<50).astype(np.uint8)
    # output_file = f'{igram_dir}/50rad_mask' 
    # driver_format = "ISCE"    # Specify the GDAL format for the output (GeoTIFF in this example)
    # rows, cols = byte_mask.shape    # Get the number of rows and columns from the NumPy array
    # data_type = gdal_array.NumericTypeCodeToGDALTypeCode(byte_mask.dtype)  # Convert NumPy data type to GDAL data type
    # driver = gdal.GetDriverByName(driver_format)
    # output_ds = driver.Create(output_file, cols, rows, 1, data_type)
    # band = output_ds.GetRasterBand(1)  # Get the single band
    # band.WriteArray(byte_mask)
    # output_ds = None

In [3]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for snaphu to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [4]:
CONFIG_TXT = f'''# snaphu configuration file
#############################################
# File input and output and runtime options #
#############################################

# Input file name
INFILE	filt_fine_aoi.int

# Input file line length 
LINELENGTH	1000

# Output file name
OUTFILE	filt_fine_aoi_noest.unw

# Correlation file name
CORRFILE	filt_fine_aoi.cor

# Coarse unwrapped-phase estimate file name (see possible file formats
# below).  The array should have the same dimensions as the input
# wrapped phase array.
#ESTIMATEFILE	veloc_unwrapped_aoi

# Input file of signed binary byte (signed char) values.  
#BYTEMASKFILE  50rad_mask

# Text file to which runtime parameters will be logged.  
LOGFILE       snaphu.logfile

# Statistical-cost mode (TOPO, DEFO, SMOOTH, or NOSTATCOSTS)
STATCOSTMODE	SMOOTH

# Algorithm used for initialization of wrapped phase values.  Possible
# values are MST and MCF.  
INITMETHOD	MCF

################
# File formats #
################

# Input file format
INFILEFORMAT		COMPLEX_DATA

# Output file format
OUTFILEFORMAT		FLOAT_DATA

# Correlation file format
CORRFILEFORMAT		FLOAT_DATA

# Unwrapped estimate file format
ESTFILEFORMAT		FLOAT_DATA

# End of snaphu configuration file'''
config_file = 'snaphu.conf.brief'

In [5]:
xml_txt = '''
<imageFile>
  <property name="WIDTH">
    <value>1000</value>
  </property>
  <property name="LENGTH">
    <value>1000</value>
  </property>
  <property name="NUMBER_BANDS">
    <value>1</value>
  </property>
  <property name="DATA_TYPE">
    <value>FLOAT</value>
  </property>
  <property name="SCHEME">
    <value>BIP</value>
  </property>
  <property name="BYTE_ORDER">
    <value>l</value>
  </property>
  <property name="ACCESS_MODE">
    <value>read</value>
  </property>
  <property name="FILE_NAME">
    <value>filt_fine_aoi_noest.unw</value>
  </property>
  <component name="Coordinate1">
    <factorymodule>isceobj.Image</factorymodule>
    <factoryname>createCoordinate</factoryname>
    <doc>First coordinate of a 2D image (width).</doc>
    <property name="name">
      <value>ImageCoordinate_name</value>
    </property>
    <property name="family">
      <value>ImageCoordinate</value>
    </property>
    <property name="size">
      <value>1000</value>
    </property>
  </component>
  <component name="Coordinate2">
    <factorymodule>isceobj.Image</factorymodule>
    <factoryname>createCoordinate</factoryname>
    <property name="name">
      <value>ImageCoordinate_name</value>
    </property>
    <property name="family">
      <value>ImageCoordinate</value>
    </property>
    <property name="size">
      <value>1000</value>
    </property>
  </component>
</imageFile>
'''
xml_file = 'filt_fine_aoi_noest.unw.xml'

In [6]:
igram_list = glob('/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interferograms/*')
veloc_dir = '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/mintpy_offsets'

In [7]:
for i, igram_dir in enumerate(igram_list):
    date = os.path.basename(igram_dir)
    print(f'working on {date}, {i+1}/{len(igram_list)}')

    tbaseline =  datetime.strptime(date[9:17], '%Y%m%d')-datetime.strptime(date[0:8], '%Y%m%d')
    print(f'temporal baseline: {tbaseline.days} days')
    
    os.chdir(igram_dir)
    #if not os.path.exists(f'{igram_dir}/filt_fine_aoi_withest.unw.xml'):
    #print('prepping inputs')
    #prep_inputs(igram_dir, veloc_dir, tbaseline.days)
    write_config_file(config_file, CONFIG_TXT, mode='w')
    print('************unwrapping************')
    !snaphu -f snaphu.conf.brief
    write_config_file(xml_file, xml_txt, mode='w')
    #else:
       # print('unwrapped igram exists, skipping')
    print('--------------------------------------------------')

working on 20191004_20191016, 1/68
temporal baseline: 12 days
write configuration to file: snaphu.conf.brief
************unwrapping************

snaphu v2.0.6
11 parameters input from file snaphu.conf.brief (52 lines total)
Logging run-time parameters to file snaphu.logfile
Reading wrapped phase from file filt_fine_aoi.int
No weight file specified.  Assuming uniform weights
Reading correlation data from file filt_fine_aoi.cor
Calculating smooth-solution cost parameters
Initializing flows with MCF algorithm
Running nonlinear network flow optimizer
Maximum flow on network: 3
Flow increment: 1  (Total improvements: 0)
Found 1 valid set(s) of connected nodes
Maximum flow on network: 1
Total solution cost: 98908689
Integrating phase
Writing output to file filt_fine_aoi_noest.unw
Program snaphu done
Elapsed processor time:   0:00:42.11
Elapsed wall clock time:  0:00:42
write configuration to file: filt_fine_aoi_noest.unw.xml
--------------------------------------------------
working on 20191